 e. Hexagons: Using the h3 and folium packages, calculate how many sizes 8 hexes contain 80% of all orders from the original data sets and visualise the hexes, colouring them by the number of fails on the map.

# Database and Setup

In [1]:
# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nbformat

# Geospatial Analysis
import geopandas as gpd
import folium
import h3

# General-purpose
import numpy as np
import scipy

#My-SQL
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='gett_taxi_database',
                                         user='root',
                                         password='admin')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
    
    
#The Query
sql_select_Query = "select * from data_orders"
cursor = connection.cursor()

# Execute the query
cursor.execute(sql_select_Query)

# Fetch the data
records = cursor.fetchall()

# Column names
column_names = [i[0] for i in cursor.description]

# Put the data into a pandas DataFrame
df = pd.DataFrame(records, columns=column_names)

# Don't forget to close the cursor and connection
cursor.close()
connection.close()

Connected to MySQL Server version  8.0.34
You're connected to database:  ('gett_taxi_database',)


# Convering Coordinates to numeric columns for calculations.

In [12]:
df['origin_latitude'] = pd.to_numeric(df['origin_latitude'], errors='coerce')
df = df.dropna(subset=['origin_latitude'])

df['origin_longitude'] = pd.to_numeric(df['origin_longitude'], errors='coerce')
df = df.dropna(subset=['origin_longitude'])

df['order_status_key'] = pd.to_numeric(df['order_status_key'], errors='coerce')

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_gk                       10716 non-null  int64  
 1   order_time                     10716 non-null  object 
 2   origin_longitude               10716 non-null  float64
 3   origin_latitude                10716 non-null  float64
 4   minutes_order_eta              2814 non-null   float64
 5   order_status_key               10716 non-null  int64  
 6   is_driver_assigned_key         10716 non-null  int64  
 7   cancellations_time_in_seconds  7307 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 753.5+ KB


## Checks (print statements)

In [13]:
print(df['origin_latitude'].min())

51.399323


In [14]:
print(df['origin_latitude'].max())

51.496169


In [15]:
print(df['origin_longitude'].min())

-1.066957


In [16]:
print(df['origin_longitude'].max())

-0.867088


In [18]:
# Check for any latitude values that are out of bounds
invalid_latitudes = df[(df['origin_latitude'] < -90) | (df['origin_latitude'] > 90)]
print(f"Invalid latitude entries: {invalid_latitudes}")

# Check for any longitude values that are out of bounds
invalid_longitudes = df[(df['origin_longitude'] < -180) | (df['origin_longitude'] > 180)]
print(f"Invalid longitude entries: {invalid_longitudes}")


Invalid latitude entries: Empty DataFrame
Columns: [order_gk, order_time, origin_longitude, origin_latitude, minutes_order_eta, order_status_key, is_driver_assigned_key, cancellations_time_in_seconds]
Index: []
Invalid longitude entries: Empty DataFrame
Columns: [order_gk, order_time, origin_longitude, origin_latitude, minutes_order_eta, order_status_key, is_driver_assigned_key, cancellations_time_in_seconds]
Index: []


 ## Question 5:
 
 Hexagons: Using the h3 and folium packages, calculate how many sizes 8 hexes contain 80% of all orders from the original data sets and visualise the hexes, colouring them by the number of fails on the map.

In [24]:
def lat_lng_to_h3(lat, lng, hex_size=8):
    return h3.geo_to_h3(lat, lng, hex_size)

In [25]:
df['hex'] = df.apply(
    lambda x: lat_lng_to_h3(x['origin_latitude'], x['origin_longitude']), axis=1)


# Print some sample data
for _, row in df.sample(5).iterrows():
    print(f"Latitude: {row['origin_latitude']}, Longitude: {row['origin_longitude']}, Hex: {row['hex']}")

Latitude: 51.471424, Longitude: -0.953193, Hex: 88195d2b51fffff
Latitude: 51.439008, Longitude: -1.058894, Hex: 88195d2963fffff
Latitude: 51.445345, Longitude: -0.947609, Hex: 88195d2b1bfffff
Latitude: 51.455172, Longitude: -0.975358, Hex: 88195d2b15fffff
Latitude: 51.442689, Longitude: -0.946847, Hex: 88195d284dfffff


In [26]:
# Assuming df is your DataFrame and it has columns 'origin_latitude' and 'origin_longitude'
# First, let's get the median values for latitude and longitude for centering the map
median_latitude = df['origin_latitude'].median()
median_longitude = df['origin_longitude'].median()
print("median_latitude", median_latitude)
print("median_longitude",median_longitude)

median_latitude 51.451972
median_longitude -0.966386


In [27]:
order_counts_per_hex = df['hex'].value_counts()
order_counts_per_hex

88195d2b1dfffff    1497
88195d2b1bfffff     870
88195d2b15fffff     774
88195d2b11fffff     707
88195d2b19fffff     667
                   ... 
88195d281dfffff       1
88195d2a1bfffff       1
88195d2a45fffff       1
88195d2b69fffff       1
88195d74d5fffff       1
Name: hex, Length: 144, dtype: int64

Color Scale

In [28]:
import branca.colormap as cm

# Calculate quantiles for color scaling
quantiles = order_counts_per_hex.quantile([0.1, 0.2, 0.4, 0.6, 0.8, 1])

# Create a custom color scale using quantiles
color_scale = cm.LinearColormap(
    colors=['green', 'yellow', 'orange', 'red', 'darkred'],
    vmin=order_counts_per_hex.min(), 
    vmax=order_counts_per_hex.max(),
    index=quantiles
)

Visualization

In [29]:
import branca.colormap as cm

# Calculate min and max values for your color scale
min_count = order_counts_per_hex.min()
max_count = order_counts_per_hex.max()

# Create a linear color scale
color_scale = cm.LinearColormap(
    colors=['green', 'yellow', 'orange', 'red', 'darkred'],
    vmin=min_count,
    vmax=max_count
)

# Make sure to normalize the scale if you have a wide range of values
color_scale = color_scale.to_step(n=5)
color_scale.caption = 'Number of Failed Orders'

# Now create the Folium map and add the hexagons
map_with_hexes = folium.Map(location=[median_latitude, median_longitude], zoom_start=12)

for hex_id, count in order_counts_per_hex.iteritems():
    hex_boundary = h3.h3_to_geo_boundary(hex_id, geo_json=True)
    hex_boundary = [(coords[1], coords[0]) for coords in hex_boundary]
    
    # Apply the color scale to each count value
    hex_color = color_scale(count)
    
    folium.Polygon(
        locations=hex_boundary,
        color=hex_color,  # The outline color of the hexagon
        fill_color=hex_color,  # The fill color of the hexagon
        fill_opacity=0.6,  # The fill opacity of the hexagon
        weight=1,  # The line weight of the hexagon
    ).add_to(map_with_hexes)

# Add the color scale to the map
color_scale.add_to(map_with_hexes)

# Display the map
map_with_hexes


In [30]:
map_with_hexes.save('Reading_Failed_Orders_Map.html')